<a href="https://colab.research.google.com/github/robertopaulo/CienciaDeDados/blob/main/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules  # Importação da função para gerar regras de associação

# Criação do dataset de exemplo (transações de supermercado)
# Cada linha representa uma transação (carrinho de compras)
# 1 = item presente, 0 = item ausente
columns = ['ID', 'Cerveja', 'Fralda', 'Bala', 'Refrigerante', 'Salgadinho']
dataset = [[1, 1, 1, 1, 1, 0],
           [2, 1, 1, 0, 0, 0],
           [3, 1, 1, 1, 0, 1],
           [4, 1, 1, 0, 1, 1],
           [5, 0, 1, 0, 1, 0],
           [6, 0, 1, 0, 0, 0],
           [7, 0, 1, 0, 0, 0],
           [8, 0, 0, 0, 1, 1],
           [9, 0, 0, 0, 1, 1]]

df = pd.DataFrame(dataset, columns=columns)

In [25]:
class Apriori:
    """Classe para encapsular a execução do algoritmo Apriori."""
    threshold = 0.5
    df = None

    def __init__(self, df, threshold=None, transform_bol=False):
        """
        Inicializa a instância do Apriori.

        Parâmetros:
        df (DataFrame): Dataset de transações (binário: 1/0)
        threshold (float): Suporte mínimo para itemsets frequentes
        transform_bol (bool): Se True, converte 1/0 para True/False
        """
        self._validate_df(df)
        self.df = df

        if threshold is not None:
            self.threshold = threshold  # Permite customizar o suporte mínimo

        if transform_bol:
            self._transform_bol()  # Conversão opcional para booleanos

    def _validate_df(self, df=None):
        """Valida se o DataFrame é válido."""
        if df is None:
            raise Exception("df deve ser um DataFrame válido do pandas.")

    def _transform_bol(self):
        """Converte representação 1/0 para True/False (requerido pelo mlxtend)."""
        for column in self.df.columns:
            self.df[column] = self.df[column].apply(lambda x: True if x == 1 else False)

    def _apriori(self, use_colnames=False, max_len=None, count=True):
        """
        Executa o algoritmo Apriori usando mlxtend.

        Parâmetros:
        use_colnames (bool): Usar nomes das colunas em vez de índices
        max_len (int): Tamanho máximo dos itemsets
        count (bool): Adicionar coluna com tamanho do itemset

        Retorna:
        DataFrame com itemsets frequentes e seus suportes
        """
        apriori_df = apriori(
            self.df,
            min_support=self.threshold,
            use_colnames=use_colnames,
            max_len=max_len
        )

        if count:
            apriori_df['length'] = apriori_df['itemsets'].apply(lambda x: len(x))

        return apriori_df

    def run(self, use_colnames=False, max_len=None, count=True):
        """Executa o Apriori e retorna os itemsets frequentes."""
        return self._apriori(
            use_colnames=use_colnames,
            max_len=max_len,
            count=count
        )

    def filter(self, apriori_df, length=None, threshold=None): #Permite filtrar por tamanho E/OU suporte (parâmetros independentes)
        """
        Filtra os itemsets frequentes por tamanho e suporte.

        Parâmetros:
        apriori_df (DataFrame): Resultado da execução do Apriori
        length (int): Tamanho específico do itemset para filtrar
        threshold (float): Valor mínimo de suporte para filtrar

        Retorna:
        DataFrame filtrado
        """
        if 'length' not in apriori_df.columns:
            raise Exception("Execute o Apriori com count=True para usar este filtro.")

        filters = []
        if length is not None:
            filters.append(apriori_df['length'] == length)
        if threshold is not None:
            filters.append(apriori_df['support'] >= threshold)

        return apriori_df[pd.concat(filters, axis=1).all(axis=1)] if filters else apriori_df

    def generate_rules(self, apriori_df, metric="confidence", min_threshold=0.7): #Nova funcionalidade para extrair regras de associação significativas
        """
        Gera regras de associação a partir dos itemsets frequentes.

        Parâmetros:
        apriori_df (DataFrame): Resultado da execução do Apriori
        metric (str): Métrica para avaliação (confidence, lift, etc.)
        min_threshold (float): Valor mínimo da métrica

        Retorna:
        DataFrame com regras de associação
        """
        return association_rules(
            apriori_df,
            metric=metric,
            min_threshold=min_threshold
        )


In [26]:
# Execução do Apriori
if 'ID' in df.columns:
    df = df.drop('ID', axis=1)  # Remove coluna ID de forma segura
    #Usando método drop() para evitar efeitos colaterais
# 1. Executa Apriori para encontrar itemsets frequentes
apriori_runner = Apriori(df, threshold=0.4, transform_bol=True)
apriori_df = apriori_runner.run(use_colnames=True)

print("\nItemsets frequentes:")
print(apriori_df)

# 2. Filtra itemsets específicos (pares com suporte > 0.41)
filtered_itemsets = apriori_runner.filter(apriori_df, length=2, threshold=0.41)
print("\nItemsets filtrados (pares com suporte > 0.41):")
print(filtered_itemsets)

# 3. Gera regras de associação (modificação nova)
rules = apriori_runner.generate_rules(
    apriori_df,
    metric="confidence",
    min_threshold=0.6  # Confiança mínima de 60%
)

# 4. Filtra regras com lift > 1 (relações significativas)
meaningful_rules = rules[rules['lift'] > 1].sort_values('confidence', ascending=False)

print("\nRegras de associação significativas (confiança > 60% e lift > 1):")
print(meaningful_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]) #Seleção inteligente de colunas:
#Mostra apenas as colunas mais relevantes para análise


Itemsets frequentes:
    support           itemsets  length
0  0.444444          (Cerveja)       1
1  0.777778           (Fralda)       1
2  0.555556     (Refrigerante)       1
3  0.444444       (Salgadinho)       1
4  0.444444  (Fralda, Cerveja)       2

Itemsets filtrados (pares com suporte > 0.41):
    support           itemsets  length
4  0.444444  (Fralda, Cerveja)       2

Regras de associação significativas (confiança > 60% e lift > 1):
  antecedents consequents   support  confidence      lift
0   (Cerveja)    (Fralda)  0.444444         1.0  1.285714
